# Setup stuff

In [151]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import csv

idr_aud_conversion_factor = 10_000
idr_aud_col_equiv = [
    ('IDR - Expenses', 'AUD - Expenses'),
    ('IDR - Income', 'AUD - Income'),
    ('IDR - Account Balance', 'AUD - Balance'),
]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [152]:
def bool_conv(x, rules):
    if x in rules:
        return rules[x]
    else:
        raise ValueError('{} not valid input for a bool conversion'.format(x))

def bool_conv_sparse(x):
    rules = {'TRUE': True, '': False}
    return bool_conv(x, rules)

def bool_conv_dense(x):
    rules = {'TRUE': True, 'FALSE': False}
    return bool_conv(x, rules)
        
def float_conv(x):
    if x == '':
        return np.nan
    else:
        try:
          return np.float(x.replace(",", ""))
        except:
          raise ValueError(x)
        
DTYPE_SPEC = {
    'Year': np.int32,
    'Month': np.int32,
    'Day': np.int32,
    'AUD - Expenses': np.float64,
    'AUD - Income': np.float64,
    'AUD - Balance': np.float64,
    # 'IDR - Expenses': np.float64,
    # 'IDR - Income': np.float64,
    # 'IDR - Account Balance': np.float64,
    'Project Codes': object,
    'Details': object,
    # 'Non-SIES': bool,
    # 'Via The SIES Account': bool,
    # 'Internal Transaction': bool,
}

CONVERTERS = {
    'Non-SIES': bool_conv_sparse,
    'Via The SIES Account': bool_conv_dense,
    'Internal Transaction': bool_conv_dense,
    'Internal': bool_conv_dense,
    'IDR - Expenses': float_conv,
    'IDR - Income': float_conv,
    'IDR - Account Balance': float_conv,
}

USE_COLS = set(DTYPE_SPEC.keys()).union(set(CONVERTERS.keys()))

def read_trans_data(dir_name, file_name):
    return pd.read_csv(
        os.path.join(dir_name, file_name),
        header=0, skiprows=(1,), thousands=',', decimal=b'.',
        usecols=lambda x: x in USE_COLS,
        true_values=('TRUE'), false_values=('FALSE'),
        parse_dates={'date': ['Year', 'Month', 'Day']},
        dtype=DTYPE_SPEC,
        converters=CONVERTERS,
    )

In [166]:
data_dir = '../../data/'
file_bang, file_indo = ('sies-bangalow-20190912-01', 'sies-indonesia-20190912-01')

bang = read_trans_data(data_dir, file_bang)
bang['Account'] = 'Bangalow'

indo = read_trans_data(data_dir, file_indo)
indo['Account'] = 'Indonesia'

indo['Non-SIES'] = (~indo['Via The SIES Account'])
indo = indo.drop('Via The SIES Account', axis=1)

indo['Internal'] = indo['Internal Transaction']
indo = indo.drop('Internal Transaction', axis=1)

for a, b in idr_aud_col_equiv:
    indo[b] = indo[a] / idr_aud_conversion_factor
    indo = indo.drop(a, axis=1)
    
full = bang.append(indo, sort=False)
# full = full.sort_values(by='date')

full = full.rename(columns={
    'AUD - Expenses': 'Expenses',
    'AUD - Income': 'Income',
    'AUD - Balance': 'Balance',
})

# print(full.dtypes)
# print(full.shape)
# print(full[(full['Expenses'].isnull() & full['Income'].isnull())])

In [185]:
def fy_summary(df):
    by_year = df.groupby(pd.DatetimeIndex(df['date']).shift(-6, freq='m').year)
    summary = by_year.sum()
    summary['Balance'] = by_year.apply(lambda ser: ser.iloc[-1,])['Balance']
    return summary
    # df.sort_index().resample("Y").apply(lambda ser: ser.iloc[-1,])['Balance']

    
account = 'Bangalow'
print('Summary by Financial Year: {} account only'.format(account))
fy_summary(full[full['Account'] == account])

Summary by Financial Year: Bangalow account only


,Expenses,Income,Balance,Non-SIES,Internal
date,,,,,
2012,-10.83,9439.00,9428.17,0.0,0.0
2013,-68962.00,79716.00,20182.17,0.0,6.0
2014,-58940.38,53652.22,14894.01,0.0,5.0
2015,-38620.00,44249.59,20523.60,1.0,4.0
2016,-98799.88,104859.76,26583.48,0.0,8.0
2017,-75010.00,68402.85,19976.33,0.0,5.0
2018,-94695.88,94591.75,19872.20,3.0,0.0


In [189]:
def project_summary(df):
    return df.groupby(df['Project Codes']).sum()
    
    
account, year = 'Bangalow', 2017
print('Summary by Project Code: financial year starting {} and {} account only'.format(year, filter))
project_summary(full[(
    (full['Account'] == filter) &
    (pd.DatetimeIndex(full['date']).shift(-6, freq='m').year == year)
)])

Summary by Project Code: financial year starting 2017 and Bangalow account only


,Expenses,Income,Balance,Non-SIES,Internal
Project Codes,,,,,
Administrative-Expense,-417.0,0.00,101175.40,0.0,0.0
Bank-Transfer-Expense,-93.0,0.00,36123.55,0.0,0.0
General-Donation,0.0,2345.00,316586.95,0.0,0.0
General-Donation-via-PayPal,0.0,16792.19,101811.97,0.0,0.0
International-Park-Tours,0.0,5200.00,45382.20,0.0,0.0
SCU-Fundraising,0.0,19852.76,136742.54,0.0,0.0
SCU-Tour-Payments,-1000.0,1600.00,36669.77,0.0,0.0
Tour-Expense-Cash-Withdrawal,-1500.0,0.00,19941.10,0.0,0.0
Transfer-to-the-Bali-Account,-40000.0,0.00,28530.08,0.0,4.0


In [190]:
year = 2017
print('Summary by Project Code: financial year starting {}, both accounts'.format(year))
full[(pd.DatetimeIndex(full['date']).shift(-6, freq='m').year == year)].groupby('Project Codes').sum()

Summary by Project Code: financial year starting 2017, both accounts


,Expenses,Income,Balance,Non-SIES,Internal
Project Codes,,,,,
Administration-Expense,-100.0000,0.00,-1307.3575,0.0,0.0
Administrative-Expense,-417.0000,0.00,101175.4000,0.0,0.0
Bank-Transfer-Expense,-93.0000,0.00,36123.5500,0.0,0.0
General-Donation,0.0000,41307.58,448536.9200,0.0,4.0
General-Donation-via-PayPal,0.0000,16792.19,101811.9700,0.0,0.0
International-Park-Tours,0.0000,5200.00,45382.2000,0.0,0.0
SCU-Fundraising,0.0000,19852.76,136742.5400,0.0,0.0
SCU-Tour-Payments,-40741.8700,44346.00,123451.1600,0.0,0.0
Tour-Expense-Cash-Withdrawal,-1500.0000,0.00,19941.1000,0.0,0.0


In [191]:
def pretty_summary(df):
    df.groupby(df['Project Codes']).sum()
    
project_summary(full)

,Expenses,Income,Balance,Non-SIES,Internal
Project Codes,,,,,
ALeRT-Tour-Donation,-20652.2500,0.0000,4.720653e+04,0.0,0.0
Administration-Expense,-609.0550,0.0000,7.063163e+04,0.0,0.0
Administrative-Expense,-854.0000,0.0000,3.305098e+05,0.0,0.0
Balance,0.0000,9718.9968,9.718997e+03,0.0,0.0
Bank-Transfer-Expense,-503.1800,0.0000,1.271047e+05,0.0,0.0
Donation-for-Tigers,0.0000,110.0000,2.052360e+04,0.0,0.0
Donation-from-Taronga,0.0000,21000.0000,4.173986e+04,0.0,0.0
Friends-of-the-National-Park-Foundation,-7505.2000,0.0000,1.425577e+04,0.0,0.0
General-Donation,-1550.0000,402019.7025,1.998760e+06,1.0,29.0
